# Module 11: SQL Basics

SQL (Structured Query Language) is how you talk to databases. Essential for working with data!

## Learning Objectives

- Understand relational databases
- Write SELECT queries with WHERE, ORDER BY, LIMIT
- Use aggregate functions and GROUP BY
- Connect to SQLite from Python
- Write queries for the Jeopardy project

---
## 1. What's a Database?

A relational database stores data in **tables** (like spreadsheets):

| id | category | value | question | answer |
|----|----------|-------|----------|--------|
| 1 | SCIENCE | 200 | This planet is red | Mars |
| 2 | HISTORY | 400 | First US president | Washington |

**Why use a database instead of just reading a JSON file?**

| JSON File | Database |
|-----------|----------|
| Load entire file into memory | Query only what you need |
| Slow for large files | Fast even with millions of rows |
| Simple for small data | Handles complex relationships |
| No built-in filtering | SQL is powerful filtering language |

For our Jeopardy game with 200K+ questions, a database is perfect!

---
## 2. SQLite: A Simple Database

SQLite is:
- Included with Python (no installation!)
- Stores the entire database in one file
- Perfect for learning and small/medium projects
- Same SQL syntax as "big" databases

In [ ]:
import sqlite3
from pathlib import Path

# Create data directory if it doesn't exist
Path("data").mkdir(exist_ok=True)

# Create a connection (creates file if doesn't exist)
conn = sqlite3.connect("data/jeopardy_practice.db")
cursor = conn.cursor()

# Create a table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS clues (
        id INTEGER PRIMARY KEY,
        category TEXT,
        value INTEGER,
        clue_text TEXT,
        answer TEXT,
        round TEXT
    )
""")
conn.commit()
print("Table created!")

In [ ]:
# Insert sample Jeopardy data
sample_clues = [
    ("SCIENCE", 200, "This planet is known as the Red Planet", "Mars", "Jeopardy!"),
    ("SCIENCE", 400, "This element has atomic number 6", "Carbon", "Jeopardy!"),
    ("SCIENCE", 600, "This force keeps planets in orbit around the Sun", "Gravity", "Jeopardy!"),
    ("SCIENCE", 800, "This subatomic particle has a positive charge", "Proton", "Jeopardy!"),
    ("SCIENCE", 1000, "This scientist developed the theory of relativity", "Einstein", "Jeopardy!"),
    ("HISTORY", 200, "The first US President", "George Washington", "Jeopardy!"),
    ("HISTORY", 400, "Year the Declaration of Independence was signed", "1776", "Jeopardy!"),
    ("HISTORY", 600, "This war ended in 1945", "World War II", "Jeopardy!"),
    ("HISTORY", 800, "This document limited the power of English kings in 1215", "Magna Carta", "Jeopardy!"),
    ("LITERATURE", 200, "Author of Romeo and Juliet", "Shakespeare", "Jeopardy!"),
    ("LITERATURE", 400, "Author of 1984", "George Orwell", "Jeopardy!"),
    ("LITERATURE", 600, "This novel begins with 'Call me Ishmael'", "Moby Dick", "Jeopardy!"),
    ("SCIENCE", 400, "This planet has the Great Red Spot", "Jupiter", "Double Jeopardy!"),
    ("SCIENCE", 800, "This element makes up most of the Sun", "Hydrogen", "Double Jeopardy!"),
]

cursor.execute("DELETE FROM clues")  # Clear existing
cursor.executemany(
    "INSERT INTO clues (category, value, clue_text, answer, round) VALUES (?, ?, ?, ?, ?)",
    sample_clues
)
conn.commit()
print(f"Inserted {len(sample_clues)} clues")

---
## 3. SELECT: Querying Data

The basic pattern:
```sql
SELECT column1, column2 FROM table_name
```

Use `*` to select all columns.

In [ ]:
# Select all columns from all rows
cursor.execute("SELECT * FROM clues")
for row in cursor.fetchall():
    print(row)

In [ ]:
# Select specific columns (cleaner output)
cursor.execute("SELECT category, value, answer FROM clues")
for row in cursor.fetchall():
    print(f"{row[0]} ${row[1]}: {row[2]}")

In [ ]:
# 🔮 Predict Before You Run
# How many rows will this return?

cursor.execute("SELECT answer FROM clues")
results = cursor.fetchall()
print(f"Number of rows: {len(results)}")

---
## 4. WHERE: Filtering Rows

```sql
SELECT * FROM table WHERE condition
```

Common operators:
- `=` equal
- `!=` or `<>` not equal
- `<`, `>`, `<=`, `>=` comparisons
- `AND`, `OR` combine conditions
- `LIKE` pattern matching

In [ ]:
# Filter by category
cursor.execute("SELECT * FROM clues WHERE category = 'SCIENCE'")
print("SCIENCE clues:")
for row in cursor.fetchall():
    print(f"  ${row[2]}: {row[4]}")

In [ ]:
# Filter by value (comparison)
cursor.execute("SELECT * FROM clues WHERE value >= 600")
print("High-value clues ($600+):")
for row in cursor.fetchall():
    print(f"  {row[1]} ${row[2]}: {row[4]}")

In [ ]:
# Multiple conditions with AND
cursor.execute("""
    SELECT * FROM clues 
    WHERE category = 'SCIENCE' AND value <= 400
""")
print("Easy SCIENCE clues:")
for row in cursor.fetchall():
    print(f"  ${row[2]}: {row[4]}")

In [ ]:
# 🔮 Predict Before You Run
# How many rows will this return?

cursor.execute("""
    SELECT * FROM clues
    WHERE category = 'SCIENCE' OR category = 'HISTORY'
""")
results = cursor.fetchall()
print(f"SCIENCE or HISTORY: {len(results)} clues")

In [ ]:
# LIKE for pattern matching (% is wildcard)
cursor.execute("""
    SELECT * FROM clues
    WHERE answer LIKE '%Washington%'
""")
print("Answers containing 'Washington':")
for row in cursor.fetchall():
    print(f"  {row[4]}")

### 🖊️ Your Turn: Write WHERE Queries

In [ ]:
# Exercise 1: Get all clues from the LITERATURE category
# YOUR QUERY HERE
cursor.execute("""  """)
lit_clues = cursor.fetchall()
print(f"Found {len(lit_clues)} LITERATURE clues")

In [ ]:
# 🧪 Grading
assert len(lit_clues) == 3, f"Should find 3 LITERATURE clues, got {len(lit_clues)}"
print("✓ Found all LITERATURE clues!")

In [ ]:
# Exercise 2: Get clues worth exactly $200 OR $1000
# YOUR QUERY HERE
cursor.execute("""  """)
extreme_values = cursor.fetchall()
print(f"Found {len(extreme_values)} clues at $200 or $1000")

In [ ]:
# 🧪 Grading
assert len(extreme_values) == 4, f"Should find 4 clues at $200 or $1000, got {len(extreme_values)}"
values = [row[2] for row in extreme_values]
assert all(v in [200, 1000] for v in values), "All should be $200 or $1000"
print("✓ Found all $200 and $1000 clues!")

---
## 5. ORDER BY and LIMIT

```sql
SELECT * FROM table ORDER BY column ASC|DESC LIMIT n
```

In [ ]:
# Sort by value (highest first)
cursor.execute("SELECT category, value, answer FROM clues ORDER BY value DESC")
print("Clues sorted by value (highest first):")
for row in cursor.fetchall():
    print(f"  ${row[1]}: {row[2]}")

In [ ]:
# Get just the top 3 highest-value clues
cursor.execute("""
    SELECT category, value, answer 
    FROM clues 
    ORDER BY value DESC 
    LIMIT 3
""")
print("Top 3 highest-value clues:")
for row in cursor.fetchall():
    print(f"  ${row[1]}: {row[2]}")

In [ ]:
# 🔮 Predict Before You Run
# What will be the FIRST row returned?

cursor.execute("""
    SELECT category, value, answer
    FROM clues
    WHERE category = 'HISTORY'
    ORDER BY value ASC
    LIMIT 1
""")
result = cursor.fetchone()
print(f"Easiest HISTORY clue: ${result[1]} - {result[2]}")

In [ ]:
# Random selection (great for games!)
cursor.execute("""
    SELECT * FROM clues
    WHERE category = 'SCIENCE' AND value = 400
    ORDER BY RANDOM()
    LIMIT 1
""")
random_clue = cursor.fetchone()
print(f"Random $400 SCIENCE clue: {random_clue[3]}")
print(f"(Run again - you might get a different one!)")

---
## 6. Aggregate Functions

Aggregate functions calculate a single value from multiple rows:

- `COUNT(*)` - count rows
- `SUM(column)` - sum values
- `AVG(column)` - average value
- `MIN(column)` - minimum value
- `MAX(column)` - maximum value

In [ ]:
# Count all clues
cursor.execute("SELECT COUNT(*) FROM clues")
total = cursor.fetchone()[0]
print(f"Total clues: {total}")

In [ ]:
# Count with a filter
cursor.execute("SELECT COUNT(*) FROM clues WHERE category = 'SCIENCE'")
science_count = cursor.fetchone()[0]
print(f"SCIENCE clues: {science_count}")

In [ ]:
# Multiple aggregates
cursor.execute("""
    SELECT 
        COUNT(*) as total,
        AVG(value) as avg_value,
        MIN(value) as min_value,
        MAX(value) as max_value
    FROM clues
""")
total, avg, min_val, max_val = cursor.fetchone()
print(f"Total: {total}")
print(f"Average value: ${avg:.0f}")
print(f"Range: ${min_val} - ${max_val}")

---
## 7. GROUP BY: Aggregate by Category

GROUP BY divides rows into groups and applies aggregates to each group.

```sql
SELECT category, COUNT(*) FROM clues GROUP BY category
```

In [ ]:
# Count clues per category
cursor.execute("""
    SELECT category, COUNT(*) as count
    FROM clues
    GROUP BY category
""")
print("Clues per category:")
for row in cursor.fetchall():
    print(f"  {row[0]}: {row[1]} clues")

In [ ]:
# Average value per category
cursor.execute("""
    SELECT category, AVG(value) as avg_value
    FROM clues
    GROUP BY category
    ORDER BY avg_value DESC
""")
print("Average value per category:")
for row in cursor.fetchall():
    print(f"  {row[0]}: ${row[1]:.0f}")

In [ ]:
# 🔮 Predict Before You Run
# Which round has more clues?

cursor.execute("""
    SELECT round, COUNT(*) as count
    FROM clues
    GROUP BY round
    ORDER BY count DESC
""")
for row in cursor.fetchall():
    print(f"{row[0]}: {row[1]} clues")

### HAVING: Filter Groups

WHERE filters rows BEFORE grouping. HAVING filters AFTER grouping.

In [ ]:
# Only show categories with more than 3 clues
cursor.execute("""
    SELECT category, COUNT(*) as count
    FROM clues
    GROUP BY category
    HAVING count > 3
""")
print("Categories with more than 3 clues:")
for row in cursor.fetchall():
    print(f"  {row[0]}: {row[1]} clues")

---
## 8. Jeopardy Project: Finding Complete Categories

For the game, we need categories that have clues at ALL five values ($200-$1000). This is a real-world query challenge!

In [ ]:
# Find categories with all 5 value levels
# We need categories where COUNT(DISTINCT value) = 5

cursor.execute("""
    SELECT category, COUNT(DISTINCT value) as value_count
    FROM clues
    WHERE round = 'Jeopardy!'
    GROUP BY category
    HAVING value_count = 5
""")

complete_categories = cursor.fetchall()
print(f"Complete categories (all 5 values): {len(complete_categories)}")
for row in complete_categories:
    print(f"  {row[0]}")

---
## 9. Using Parameters (Preventing SQL Injection)

**Never** put user input directly into SQL strings!

```python
# BAD - SQL injection risk!
cursor.execute(f"SELECT * FROM clues WHERE category = '{user_input}'")

# GOOD - parameterized query
cursor.execute("SELECT * FROM clues WHERE category = ?", (user_input,))
```

In [ ]:
# Safe parameterized query
def get_clues_by_category(category_name):
    cursor.execute("""
        SELECT value, clue_text, answer 
        FROM clues 
        WHERE category = ?
        ORDER BY value
    """, (category_name,))
    return cursor.fetchall()

# Use it
science_clues = get_clues_by_category("SCIENCE")
print(f"Found {len(science_clues)} SCIENCE clues")
for value, clue, answer in science_clues:
    print(f"  ${value}: {answer}")

In [ ]:
# Query with multiple parameters
def get_random_clue(category, value):
    cursor.execute("""
        SELECT clue_text, answer
        FROM clues
        WHERE category = ? AND value = ?
        ORDER BY RANDOM()
        LIMIT 1
    """, (category, value))
    return cursor.fetchone()

clue = get_random_clue("HISTORY", 400)
if clue:
    print(f"Clue: {clue[0]}")
    print(f"Answer: {clue[1]}")

---
## 10. Practice: Write These Queries

In [ ]:
# 🖊️ Exercise 1: Get all clues worth $200
# Store results in query1_results

cursor.execute("""  """)
query1_results = cursor.fetchall()
print(f"Found {len(query1_results)} clues worth $200")

In [ ]:
# 🧪 Grading
assert len(query1_results) == 3, f"Should find 3 clues worth $200, got {len(query1_results)}"
assert all(row[2] == 200 for row in query1_results), "All results should have value=200"
print("✓ Query 1 correct!")

In [ ]:
# 🖊️ Exercise 2: Count clues in the LITERATURE category
# Store the count in query2_count

cursor.execute("""  """)
query2_count = cursor.fetchone()[0]
print(f"LITERATURE has {query2_count} clues")

In [ ]:
# 🧪 Grading
assert query2_count == 3, f"Should count 3 LITERATURE clues, got {query2_count}"
print("✓ Query 2 correct!")

In [ ]:
# 🖊️ Exercise 3: Get the single highest-value clue
# Store the result in query3_result

cursor.execute("""  """)
query3_result = cursor.fetchone()
print(f"Highest value clue: ${query3_result[2]} - {query3_result[4]}")

In [ ]:
# 🧪 Grading
assert query3_result[2] == 1000, f"Highest value should be 1000, got {query3_result[2]}"
print("✓ Query 3 correct!")

In [ ]:
# 🖊️ Exercise 4: Find the category with the most clues
# Hint: GROUP BY, ORDER BY, and LIMIT
# Store the category name in query4_category

cursor.execute("""  """)
result = cursor.fetchone()
query4_category = result[0]
query4_count = result[1]
print(f"Most clues: {query4_category} with {query4_count} clues")

In [ ]:
# 🧪 Grading
assert query4_category == "SCIENCE", f"Should be SCIENCE, got {query4_category}"
assert query4_count == 7, f"SCIENCE should have 7 clues, got {query4_count}"
print("✓ Query 4 correct!")

In [ ]:
# 🖊️ Exercise 5: Get a random HISTORY clue at $400
# Use ORDER BY RANDOM() and LIMIT 1
# Store the clue text in query5_clue

cursor.execute("""  """)
result = cursor.fetchone()
query5_clue = result[3]  # clue_text column
print(f"Random HISTORY $400 clue: {query5_clue}")

In [ ]:
# 🧪 Grading
# There's only one $400 HISTORY clue in our sample
assert "Declaration" in query5_clue or "1776" in query5_clue, "Should be the Declaration clue"
print("✓ Query 5 correct!")

---
## 11. SQL Query Cheat Sheet

```sql
-- Basic SELECT
SELECT * FROM clues
SELECT category, value FROM clues

-- Filtering
SELECT * FROM clues WHERE category = 'SCIENCE'
SELECT * FROM clues WHERE value >= 400
SELECT * FROM clues WHERE category = 'SCIENCE' AND value <= 400
SELECT * FROM clues WHERE value = 200 OR value = 1000
SELECT * FROM clues WHERE answer LIKE '%Washington%'

-- Sorting and limiting
SELECT * FROM clues ORDER BY value DESC
SELECT * FROM clues ORDER BY value DESC LIMIT 5
SELECT * FROM clues ORDER BY RANDOM() LIMIT 1

-- Aggregates
SELECT COUNT(*) FROM clues
SELECT AVG(value), MIN(value), MAX(value) FROM clues

-- Grouping
SELECT category, COUNT(*) FROM clues GROUP BY category
SELECT category, COUNT(*) FROM clues GROUP BY category HAVING COUNT(*) > 3

-- Parameterized (in Python)
cursor.execute("SELECT * FROM clues WHERE category = ?", (category,))
```

In [ ]:
# Clean up
conn.close()
print("Database connection closed.")

---
## Key Takeaways

1. **SELECT gets data** - `SELECT columns FROM table`
2. **WHERE filters rows** - `WHERE condition`
3. **ORDER BY sorts** - `ORDER BY column DESC/ASC`
4. **LIMIT restricts count** - `LIMIT n`
5. **GROUP BY aggregates** - Combine with COUNT, AVG, etc.
6. **Use parameterized queries** - Never put user input directly in SQL!
7. **ORDER BY RANDOM()** - Perfect for game random selection

### For the Jeopardy Project

Key queries you'll need:
- Get random clue by category and value
- Find categories with all 5 value levels
- Count clues per category
- List all unique categories

---

**Congratulations!** You've completed the fundamentals. Time to build the Jeopardy game!